In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from darts import TimeSeries

from aare.constants import LOC_BERN, LOC_THUN, TIME, TEMP
from aare.remote_existenz_store import RemoteExistenzStore

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
store = RemoteExistenzStore()

In [ ]:
freq = "1h"

In [ ]:
df = store.query_hydro("0", LOC_BERN, agg_freq=freq)
df

In [ ]:
px.line(df, TIME, TEMP)

In [ ]:
df.loc[df["temperature"] <= 0] = np.nan

In [ ]:
px.line(df, TIME, TEMP)

In [ ]:
df = df.set_index(TIME)
# turn it into timezone-naive timestamps because that's what darts needs
df.index = df.index.tz_localize(None)
# kill na, also kills NaT
df = df.dropna()

In [ ]:
df

In [ ]:
ts = TimeSeries.from_times_and_values(df.index, df[TEMP], freq="h", fill_missing_dates=True)
# ts = TimeSeries.from_dataframe(df, freq=freq, fill_missing_dates=True, fillna_value=True)
ts

In [ ]:
ts.gaps()